## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

Кількість користувачів, яку треба зібрати у кожній групі, визначається аналізом потужності. Вона залежить від:

* статистичної потужності (0.8) - беремо "стандарт" ,
* рівня значущості (alpha = 0.05) - беремо звичайний "стандарт",
* очікуваної величини ефекту.

In [25]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
from statsmodels.stats import weightstats as stests
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

%matplotlib inline

In [ ]:
# Розрахунок розміру ефекту на основі наших очікуваних показників
effect_size = sms.proportion_effectsize(0.2, 0.19)

In [ ]:
effect_size.round(6)

np.float64(0.025242)

In [ ]:
# Розрахунок необхідного розміру вибірки
required_n = sms.NormalIndPower().solve_power(
    effect_size,
    power=0.8,
    alpha=0.05,
    ratio=1
    )
# Округлення до наступного цілого числа
required_n = ceil(required_n)

print('розмір вибірки:', required_n)

розмір вибірки: 24638


Нам буде потрібно мінімум **24 638** спостережень для кожної групи.

Power = 0.8 означає, що якщо реальна різниця в утриманні дійсно становить той самий 1% (19% проти 20%), то з розрахованим розміром вибірки ми маємо 80% шанс виявити цю різницю, як статистично значущу.

2. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

гіпотеза

Но: утримання однакове для gate_30 і gate_40

На: є різниця утримання для gate_30 і gate_40

In [11]:
df_cats = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/Data/cookie_cats.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
df_cats.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [14]:
df_cats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [22]:
df_cats.groupby('version')['retention_7'].mean()

,retention_7
version,
gate_30,0.190201
gate_40,0.182000


Дивлячись на середне значення по retention_7 можна сказати, що утримання майже однакове і для gate_30, і для gate_40. Трохи краще у gate_30. Але це треба більш детально перевіряти, бо цифри майже однакові

3. Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


In [67]:
calc = df_cats.groupby('version')['retention_7']
calc = calc.agg(['mean', 'std', 'count'])
calc.columns = ['mean', 'std', 'count']

calc

,mean,std,count
version,,,
gate_30,0.190201,0.392464,44700
gate_40,0.182000,0.385849,45489


In [68]:
gate_30 = df_cats[df_cats['version'] == 'gate_30']['retention_7']
gate_40 = df_cats[df_cats['version'] == 'gate_40']['retention_7']

In [77]:
count_30 = gate_30.count()
count_40 = gate_40.count()
true_result = [gate_30.sum(), gate_40.sum()]
nobs = [count_30, count_40]

In [78]:
true_result

[np.int64(8502), np.int64(8279)]

In [79]:
nobs

[np.int64(44700), np.int64(45489)]

In [80]:
z_stat, p_val = proportions_ztest(true_result, nobs=nobs)
(lower_30, lower_40), (upper_30, upper_40) = proportion_confint(true_result, nobs=nobs, alpha=0.05)

print('Z - статистика:', z_stat.round(4))
print('P - value:', p_val.round(4))
print(f'Довірчий інтервал 95% для групи control: [{lower_30:.3f}, {upper_30:.3f}]')
print(f'Довірчий інтервал 95% для групи treatment: [{lower_40:.3f}, {upper_40:.3f}]')

Z - статистика: 3.1644
P - value: 0.0016
Довірчий інтервал 95% для групи control: [0.187, 0.194]
Довірчий інтервал 95% для групи treatment: [0.178, 0.186]


1. так як P - value = 0,0016 < 0,05, то різниця є статистично значущою і ми відхиляємо Но, тобто утримання для різних воріт не однакове

2. Довірчі інтервали не перетинаються, так для gate_30 утримання гравців в інтервалі 18,7 - 19,4%, в той час як для gate_40 - це 17,8-18,6%. Тобто переміщення воріт погіршило ситуацію.

4. Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.


In [88]:
cats_xi = pd.crosstab(df_cats['version'], df_cats['retention_7'])

In [89]:
chi2, p, dof, expected = stats.chi2_contingency(cats_xi)

print('χ² =', chi2.round(4))
print('P - value:', p.round(4))
print(f"Ступені свободи = {dof}")
print("Очікувані частоти:\n", expected.round (4))

χ² = 9.9591
P - value: 0.0016
Ступені свободи = 1
Очікувані частоти:
 [[36382.9026  8317.0974]
 [37025.0974  8463.9026]]


1. гіпотезу залишаю таку ж саму, як і в z-тесті:

гіпотеза

Но: утримання однакове для gate_30 і gate_40

На: є різниця утримання для gate_30 і gate_40

2. Висновок співпадає з висновком з z-тесту:
      P - value=0.0016< 0.05, відхиляємо нульову гіпотезу


По факту перехід до gate_40 не привів до плануємого результату, і кращі результати утримання в gate_30

На рівні значущості 5% переміщення воріт на 40-й рівень негативно вплинуло на утримання, так як воно знизилось з майже 19% (gate_30) до 18,2% (gate_40)